In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

import gc

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection, preprocessing, metrics

import lightgbm as lgb

from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
gc.collect()

280

In [3]:
campaign = pd.read_csv('campaign_data.csv')
coupon = pd.read_csv('coupon_item_mapping.csv')
item = pd.read_csv('item_data.csv')
customer = pd.read_csv('customer_demographics.csv')
txn = pd.read_csv('customer_transaction_data.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
campaign['start_date'] = pd.to_datetime(campaign.start_date, format = "%d/%m/%y")
campaign['end_date'] = pd.to_datetime(campaign.end_date, format = "%d/%m/%y")
txn['date'] = pd.to_datetime(txn.date, format = "%Y-%m-%d")

In [5]:
txn = pd.merge(txn, item, on = 'item_id', how = 'left')
txn = pd.merge(txn, customer, on = 'customer_id', how = 'left')
coupon = pd.merge(coupon, item, on = 'item_id', how ='left')

In [6]:
txn['week_day'] = txn.date.dt.weekday
txn['month_day'] = txn.date.dt.day
txn['month'] = txn.date.dt.month
txn['year'] = txn.date.dt.year
txn['week'] = txn.date.dt.week
txn['weekend'] = np.where(txn['week_day']>4, "weekend", "weekday")
txn['total_discount'] = txn.other_discount + txn.coupon_discount
txn['mrp'] = txn.other_discount + txn.selling_price + txn.coupon_discount
txn['other_discount_per'] = txn.other_discount/txn.mrp
txn['coupon_discount_per'] = txn.coupon_discount/txn.mrp
txn['total_discount_per'] = txn.total_discount/txn.mrp

In [7]:
txn['income_bracket'] = txn.income_bracket.fillna(0).astype(int).astype(str)
txn['rented'] = txn.rented.fillna(2).astype(int).astype(str)

In [8]:
campaign_coupon = train[['campaign_id', 'coupon_id']].drop_duplicates().append(test[['campaign_id', 'coupon_id']].drop_duplicates())

In [9]:
def campaign_overlap(campaign):
    campaign_overlap = pd.DataFrame()
    for c in campaign.campaign_id:
        start_date = campaign[campaign.campaign_id==c].iloc[0]['start_date']
        end_date = campaign[campaign.campaign_id==c].iloc[0]['end_date']
        temp = campaign[((campaign.start_date>=start_date) & (campaign.end_date<=end_date))
                       | ((campaign.start_date>=start_date) & (campaign.start_date<=end_date))
                       | ((campaign.start_date<=start_date) & (campaign.end_date>=start_date))]
        temp['for_campaign'] = c
        campaign_overlap = temp.append(campaign_overlap)
    campaign_overlap = campaign_overlap[campaign_overlap.campaign_id != campaign_overlap.for_campaign]
    campaign_overlap.columns = ['overlap_campaign', 'overlap_campaign_type', 'overlap_start_date', 'overlap_end_date', 'campaign_id']
    campaign_overlap = pd.merge(campaign_overlap, campaign, on = 'campaign_id', how = 'left')
    
    campaign_overlap['max_start_date'] = np.where(campaign_overlap.start_date > campaign_overlap.overlap_start_date, 
             campaign_overlap.start_date, campaign_overlap.overlap_start_date)

    campaign_overlap['min_end_date'] = np.where(campaign_overlap.end_date < campaign_overlap.overlap_end_date, 
             campaign_overlap.end_date, campaign_overlap.overlap_end_date)
    
    campaign_overlap['overlap_days'] = (campaign_overlap['min_end_date'] - campaign_overlap['max_start_date'])/np.timedelta64(1,'D')
    
    overlap_count = pd.pivot_table(campaign_overlap, values = 'overlap_campaign', index = 'campaign_id', columns = 'overlap_campaign_type', aggfunc= 'nunique', fill_value=0).reset_index()
    overlap_count.columns = ['campaign_id', 'x_overlap_count', 'y_overlap_count']
    
    overlap_days = pd.pivot_table(campaign_overlap, values = 'overlap_days', index = 'campaign_id', columns = 'overlap_campaign_type', aggfunc= 'max', fill_value=0).reset_index()
    overlap_days.columns = ['campaign_id', 'x_max_overlap_days', 'y_max_overlap_days']
    
    campaign_overlap_features = pd.merge(overlap_count, overlap_days, on = 'campaign_id', how = 'outer')
    
    return campaign_overlap_features
    

In [10]:
def customer_item_txn(txn):
    
    features = txn.groupby(['customer_id','item_id'])['quantity','selling_price', 'other_discount','coupon_discount'].sum().reset_index()
    features.columns = ['customer_id','item_id', 'total_quantity','gmv', 'total_other_discount','total_coupon_discount']
    
    disc_count_features = txn[txn.coupon_discount!=0].groupby(['customer_id','item_id'])['coupon_discount'].count().reset_index()
    disc_count_features.columns = ['customer_id','item_id','coupon_used']
    
    order_count = txn.groupby(['customer_id','item_id'])['date'].count().reset_index()
    order_count.columns = ['customer_id','item_id','order_count']
    
    features = pd.merge(pd.merge(features, 
                                 disc_count_features, on = ['customer_id','item_id'], how = 'left'), 
                        order_count, on = ['customer_id','item_id'], how = 'left').fillna(0)
    #features = features['coupon_used']/features['order_count']
    
    return features

In [11]:
def customer_campaign_coupon(txn, coupon):
    
    customer_item_features = customer_item_txn(txn)
    df = pd.merge(customer_item_features, coupon, on = 'item_id', how = 'left')
    df = df[df.coupon_id.notnull()]
    df['coupon_id'] = df['coupon_id'].astype(int)
    
    id_cols = ['customer_id', 'coupon_id']
    
    unique_count_item = pd.pivot_table(df, values='item_id', index=['customer_id', 'coupon_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index()
    unique_count_item.columns = ['customer_id',  'coupon_id', 'ccc_Established_item_cnt', 'ccc_Local_item_cnt']
    unique_count_item['ccc_Established_item_cnt_per'] = unique_count_item['ccc_Established_item_cnt']/(unique_count_item['ccc_Established_item_cnt']+unique_count_item['ccc_Local_item_cnt'])
    
    unique_count_brand = pd.pivot_table(df, values='brand', index=['customer_id', 'coupon_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index()
    unique_count_brand.columns = ['customer_id', 'coupon_id', 'ccc_Established_brand_cnt', 'ccc_Local_brand_cnt']
    
    unique_count_order = pd.pivot_table(df, values='order_count', index=['customer_id',  'coupon_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index()
    unique_count_order.columns = ['customer_id', 'coupon_id', 'ccc_Established_order_cnt', 'ccc_Local_order_cnt']
    
    unique_count_coupon = pd.pivot_table(df, values='coupon_used', index=['customer_id', 'coupon_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    unique_count_coupon.columns = ['customer_id', 'coupon_id', 'ccc_Established_coupon_used', 'ccc_Local_coupon_used']
    
    
    sum_gmv = pd.pivot_table(df, values='gmv', index=['customer_id', 'coupon_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    sum_gmv.columns = ['customer_id', 'coupon_id', 'ccc_Established_gmv', 'ccc_Local_gmv']
    
    sum_other_discount = pd.pivot_table(df, values='total_other_discount', index=['customer_id', 'coupon_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    sum_other_discount.columns = ['customer_id', 'coupon_id', 'ccc_Established_other_disc', 'ccc_Local_other_disc']
    
    sum_coupon_discount = pd.pivot_table(df, values='total_coupon_discount', index=['customer_id', 'coupon_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    sum_coupon_discount.columns = ['customer_id', 'coupon_id', 'ccc_Established_coupon_disc', 'ccc_Local_coupon_disc']
    
    order_qty_category = pd.pivot_table(df, values='total_quantity', index=['customer_id', 'coupon_id'], columns='category', aggfunc=np.sum, fill_value=0).reset_index()
    order_qty_category['total_quantity'] = order_qty_category.drop(columns = ['customer_id', 'coupon_id']).sum(axis=1)
    order_qty_category = pd.concat([order_qty_category[['customer_id', 'coupon_id', 'total_quantity']], order_qty_category.drop(columns = ['customer_id', 'coupon_id', 'total_quantity']).div(order_qty_category['total_quantity']/100, axis=0)], axis = 1)
    order_qty_category.columns = ['customer_id', 'coupon_id', 'ccc_total_qty'] + list(str('ccc_qty_per_') + order_qty_category.columns[3:])
    
    features = pd.concat([unique_count_item, unique_count_brand.drop(columns = id_cols), 
                          unique_count_order.drop(columns = id_cols), unique_count_coupon.drop(columns = id_cols),
                          sum_gmv.drop(columns = id_cols), sum_other_discount.drop(columns = id_cols), 
                          sum_coupon_discount.drop(columns = id_cols), order_qty_category.drop(columns = id_cols)], axis = 1)
    
    features['ccc_Established_coupon_affinity'] = features['ccc_Established_coupon_used']/features['ccc_Established_order_cnt']
    features['ccc_Local_coupon_affinity'] = features['ccc_Local_coupon_used']/features['ccc_Local_order_cnt']
    
    features['ccc_Established_other_disc_per'] = features['ccc_Established_other_disc']/(features['ccc_Established_other_disc'] + features['ccc_Established_coupon_disc'] + features['ccc_Established_gmv'])
    features['ccc_Established_coupon_disc_per'] = features['ccc_Established_coupon_disc']/(features['ccc_Established_other_disc'] + features['ccc_Established_coupon_disc'] + features['ccc_Established_gmv'])
    
    features['ccc_Local_other_disc_per'] = features['ccc_Local_other_disc']/(features['ccc_Local_other_disc'] + features['ccc_Local_coupon_disc'] + features['ccc_Local_gmv'])
    features['ccc_Local_coupon_disc_per'] = features['ccc_Local_coupon_disc']/(features['ccc_Local_other_disc'] + features['ccc_Local_coupon_disc'] + features['ccc_Local_gmv'])
    
    return features.fillna(0)

In [12]:
def customer_txn(txn):
    
    id_cols = ['customer_id']
    
    unique_count_item = pd.pivot_table(txn, values='item_id', index=['customer_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index()
    unique_count_item.columns = ['customer_id', 'cx_Established_item_cnt', 'cx_Local_item_cnt']
    unique_count_item['cx_Established_item_cnt_per'] = unique_count_item['cx_Established_item_cnt']/(unique_count_item['cx_Established_item_cnt']+unique_count_item['cx_Local_item_cnt'])
    
    unique_count_order = pd.pivot_table(txn, values='date', index=['customer_id'], columns='brand_type', aggfunc='count', fill_value=0).reset_index()
    unique_count_order.columns = ['customer_id', 'cx_Established_order_cnt', 'cx_Local_order_cnt']

    
    unique_count_brand = pd.pivot_table(txn, values='brand', index=['customer_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index()
    unique_count_brand.columns = ['customer_id', 'cx_Established_brand_cnt', 'cx_Local_brand_cnt']
    
    count_coupon_used = pd.pivot_table(txn[txn.coupon_discount!=0], values='coupon_discount', index=['customer_id'], columns='brand_type', aggfunc='count', fill_value=0).reset_index()
    count_coupon_used.columns = ['customer_id'] + list(str('cx_') + count_coupon_used.columns[1:] + str('_coupon_used'))
    #'cx_Established_coupon_used', 'cx_Local_coupon_used']
    
    sum_qty_weekday = pd.pivot_table(txn, values='quantity', index=['customer_id'], columns='weekend', aggfunc=np.sum, fill_value=0).reset_index()
    sum_qty_weekday.columns = ['customer_id', 'cx_weekday_qty_cnt', 'cx_weekend_qty_cnt']
    sum_qty_weekday['cx_weekend_affinity'] = sum_qty_weekday['cx_weekend_qty_cnt']/(sum_qty_weekday['cx_weekday_qty_cnt']+sum_qty_weekday['cx_weekend_qty_cnt'])
    
    sum_gmv = pd.pivot_table(txn, values='selling_price', index=['customer_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    sum_gmv.columns = ['customer_id', 'cx_Established_gmv', 'cx_Local_gmv']
    sum_gmv['cx_Established_gmv_per'] = sum_gmv['cx_Established_gmv']/(sum_gmv['cx_Established_gmv']+sum_gmv['cx_Local_gmv'])
    
    sum_other_discount = pd.pivot_table(txn, values='other_discount', index=['customer_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    sum_other_discount.columns = ['customer_id', 'cx_Established_other_disc', 'cx_Local_other_disc']
    
    sum_coupon_discount = pd.pivot_table(txn, values='coupon_discount', index=['customer_id'], columns='brand_type', aggfunc=np.sum, fill_value=0).reset_index()
    sum_coupon_discount.columns = ['customer_id', 'cx_Established_coupon_disc', 'cx_Local_coupon_disc']
    
    order_qty_category = pd.pivot_table(txn, values='quantity', index=['customer_id'], columns='category', aggfunc=np.sum, fill_value=0).reset_index()
    order_qty_category['total_quantity'] = order_qty_category.drop(columns = ['customer_id']).sum(axis=1)
    order_qty_category = pd.concat([order_qty_category[['customer_id', 'total_quantity']], order_qty_category.drop(columns = ['customer_id', 'total_quantity']).div(order_qty_category['total_quantity']/100, axis=0)], axis = 1)
    order_qty_category.columns = ['customer_id', 'cx_total_qty'] + list(str('cx_qty_per_') + order_qty_category.columns[2:])
    
    first_txn = txn.groupby('customer_id')['date'].agg(['min','max']).reset_index()
    first_txn.columns = ['customer_id', 'first_txn_date', 'last_txn_date']
    
    features = pd.concat([unique_count_item, sum_qty_weekday.drop(columns = id_cols), 
                          unique_count_brand.drop(columns = id_cols), sum_gmv.drop(columns = id_cols), 
                          sum_other_discount.drop(columns = id_cols), sum_coupon_discount.drop(columns = id_cols), 
                          order_qty_category.drop(columns = id_cols), first_txn.drop(columns = id_cols),
                         count_coupon_used.drop(columns = id_cols), unique_count_order.drop(columns = id_cols)], axis = 1)
    
    features['cx_Established_coupon_affinity'] = features['cx_Established_coupon_used']/features['cx_Established_order_cnt']
    features['cx_Local_coupon_affinity'] = features['cx_Local_coupon_used']/features['cx_Local_order_cnt']
    
    features['cx_Established_other_disc_per'] = features['cx_Established_other_disc']/(features['cx_Established_other_disc'] + features['cx_Established_coupon_disc'] + features['cx_Established_gmv'])
    features['cx_Established_coupon_disc_per'] = features['cx_Established_coupon_disc']/(features['cx_Established_other_disc'] + features['cx_Established_coupon_disc'] + features['cx_Established_gmv'])
    
    features['cx_Local_other_disc_per'] = features['cx_Local_other_disc']/(features['cx_Local_other_disc'] + features['cx_Local_coupon_disc'] + features['cx_Local_gmv'])
    features['cx_Local_coupon_disc_per'] = features['cx_Local_coupon_disc']/(features['cx_Local_other_disc'] + features['cx_Local_coupon_disc'] + features['cx_Local_gmv'])
    
    return features.fillna(0)   

In [13]:
def item_txn(txn):
    sum_features = txn.groupby('item_id')['selling_price', 'quantity','other_discount', 'coupon_discount'].sum().reset_index()
    sum_features.columns = ['item_id', 'selling_price', 'quantity','other_discount', 'coupon_discount']
    sum_features['avg_coupon_discount_per'] = sum_features['coupon_discount']/(sum_features['coupon_discount'] + sum_features['other_discount'] + sum_features['selling_price'])
    sum_features['avg_other_discount_per'] = sum_features['other_discount']/(sum_features['coupon_discount'] + sum_features['other_discount'] + sum_features['selling_price'])
    
    order_count = txn.groupby('item_id')['date'].count().reset_index()
    order_count.columns = ['item_id', 'orders']
    
    unique_customer_count = txn.groupby('item_id')['customer_id'].count().reset_index()
    unique_customer_count.columns = ['item_id', 'unique_customer_id']
    
    coupon_discount_count = txn[txn.coupon_discount!=0].groupby('item_id')['coupon_discount'].count().reset_index()
    coupon_discount_count.columns = ['item_id', 'coupon_discount_used']
    
    features = pd.concat([sum_features, order_count.drop(columns = ['item_id']), 
               unique_customer_count.drop(columns = ['item_id']), coupon_discount_count.drop(columns = ['item_id'])], axis = 1).fillna(0)
    
    return features

In [14]:
def campaign_coupon_feature(coupon, txn):
    
    df = pd.merge(txn, coupon[['coupon_id','item_id']], on = 'item_id', how = 'left')
    df = pd.merge(df, customer, on = 'customer_id', how = 'left')
    id_cols = ['coupon_id']
    
    unique_item_count = pd.pivot_table(df, values = 'item_id', index=['coupon_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index().fillna(0)
    unique_item_count.columns = ['coupon_id', 'cc_item_count_Estabilished', 'cc_item_count_Local']
    
    unique_brand_count = pd.pivot_table(df, values = 'brand', index=['coupon_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index().fillna(0)
    unique_brand_count.columns = ['coupon_id', 'cc_brand_count_Estabilished', 'cc_brand_count_Local']
    
    mode_category = pd.pivot_table(df, values = 'category', index=['coupon_id'], columns='brand_type', aggfunc=lambda x:x.value_counts().index[0], fill_value=0).reset_index()
    mode_category.columns = ['coupon_id', 'cc_frequent_cat_Estabilished', 'cc_frequent_cat_Local']
    
    avg_selling_price = pd.pivot_table(df, values = 'selling_price', index=['coupon_id'], columns='brand_type', aggfunc='mean', fill_value=0).reset_index().fillna(0)
    avg_selling_price.columns = ['coupon_id', 'cc_avg_asp_Estabilished', 'cc_avg_asp_Local']
    
    avg_coupon_discount = pd.pivot_table(df, values = 'coupon_discount', index=['coupon_id'], columns='brand_type', aggfunc='mean', fill_value=0).reset_index().fillna(0)
    avg_coupon_discount.columns = ['coupon_id', 'cc_coupon_discount_Estabilished', 'cc_coupon_discount_Local']
    
    unique_customers_count = pd.pivot_table(df, values = 'customer_id', index=['coupon_id'], columns='brand_type', aggfunc='nunique', fill_value=0).reset_index().fillna(0)
    unique_customers_count.columns = ['coupon_id', 'cc_customers_Estabilished', 'cc_customers_Local']
    
    coupon_used = pd.pivot_table(df[df.coupon_discount!=0], values = 'coupon_discount', index=['coupon_id'], columns='brand_type', aggfunc='count', fill_value=0).reset_index().fillna(0)
    #coupon_used.columns = ['coupon_id', 'cc_coupon_used_Estabilished', 'cc_coupon_used_Local']
    coupon_used.columns = ['coupon_id'] + list(str('cc_coupon_used_') + coupon_used.columns[1:] )
    
    #mode_age = pd.pivot_table(df, values = 'age_range', index=['coupon_id'], columns='brand_type', aggfunc=lambda x:x.value_counts().index[0], fill_value=0).reset_index()
    #mode_age.columns = ['coupon_id', 'cc_frequent_age_Estabilished', 'cc_frequent_age_Local']
    
    #mode_gender = pd.pivot_table(df, values = 'gender', index=['coupon_id'], columns='brand_type', aggfunc=lambda x:x.value_counts().index[0], fill_value=0).reset_index()
    #mode_gender.columns = ['coupon_id', 'cc_frequent_gender_Estabilished', 'cc_frequent_gender_Local']
    
    #mode_marital_status = pd.pivot_table(df, values = 'marital_status', index=['coupon_id'], columns='brand_type', aggfunc=lambda x:x.value_counts().index[0], fill_value=0).reset_index()
    #mode_marital_status.columns = ['coupon_id', 'cc_frequent_ms_Estabilished', 'cc_frequent_ms_Local']
    
    #mode_child = pd.pivot_table(df, values = 'no_of_children', index=['coupon_id'], columns='brand_type', aggfunc=lambda x:x.value_counts().index[0], fill_value=0).reset_index()
    #mode_child.columns = ['coupon_id', 'cc_frequent_child_Estabilished', 'cc_frequent_child_Local']
    
    #mode_income = pd.pivot_table(df, values = 'income_bracket', index=['coupon_id'], columns='brand_type', aggfunc=lambda x:x.value_counts().index[0], fill_value=0).reset_index()
    #mode_income.columns = ['coupon_id', 'cc_frequent_income_Estabilished', 'cc_frequent_income_Local']
    
    features = pd.concat([unique_item_count, unique_brand_count.drop(columns = id_cols), 
                          mode_category.drop(columns = id_cols), avg_selling_price.drop(columns = id_cols), 
                          avg_coupon_discount.drop(columns = id_cols), unique_customers_count.drop(columns = id_cols),
                         coupon_used.drop(columns = id_cols)#, mode_age.drop(columns = id_cols), 
                          #mode_marital_status.drop(columns = id_cols), 
                          #mode_child, mode_income.drop(columns = id_cols)
                         ], axis = 1)
    
    return features

In [15]:
def txn_features(txn, campaign, coupon):
    ccc_features = pd.DataFrame()
    cx_features = pd.DataFrame()
    cc_features = pd.DataFrame()

    for c in campaign.campaign_id.unique():
        txn2 = txn
        start_date = campaign[campaign.campaign_id==c].iloc[0]['start_date']
        end_date = campaign[campaign.campaign_id==c].iloc[0]['end_date']
        
        #txn2 = txn2[(txn2.date < (start_date - datetime.timedelta(days=110))) & (txn2.date > (start_date - datetime.timedelta(days=130)))]
        txn2 = txn2[(txn2.date < (start_date - datetime.timedelta(days=110)))]
        #txn2 = txn2[(txn2.date < start_date) & (txn2.date > (start_date - datetime.timedelta(days=220)))]
        #txn2 = txn2[(txn2.date < start_date)]
        
        c_c_c_features = customer_campaign_coupon(txn2, coupon)
        c_c_c_features['campaign_id'] = c
        ccc_features = c_c_c_features.append(ccc_features)
        
        c_features = customer_txn(txn2)
        c_features['Vintage'] = (start_date - c_features.first_txn_date)/np.timedelta64(1,'D')
        c_features['days_since_last_txn'] = (start_date - c_features.last_txn_date)/np.timedelta64(1,'D')
        c_features['campaign_id'] = c
        cx_features = c_features.append(cx_features)
        
        #c_c_features = campaign_coupon_feature(coupon, item_txn(txn2))
        c_c_features = campaign_coupon_feature(coupon, txn2)
        c_c_features['campaign_id'] = c
        cc_features = c_c_features.append(cc_features)
    
    return ccc_features, cx_features, cc_features

In [16]:
def campaign_feature(campaign, campaign_coupon_item):
    
    campaign['campaign_days'] = (campaign.end_date - campaign.start_date)/np.timedelta64(1,'D')
    campaign['campaign_weekends'] = [pd.date_range(x,y).weekday.isin([5,6]).sum() for x , y in zip(campaign.start_date,campaign.end_date)]
    campaign['campaign_start_month'] = campaign.start_date.dt.month
    
    count_features = campaign_coupon_item.groupby('campaign_id')['coupon_id','item_id','brand','category'].nunique().reset_index()
    count_features.columns = ['campaign_id', 'campaign_unique_coupon_cnt', 'campaign_unique_item_cnt', 'campaign_unique_brand_cnt', 'campaign_unique_category_cnt']
    
    mode_features = campaign_coupon_item.groupby(['campaign_id'])['coupon_id','item_id','brand','category'].agg(lambda x:x.value_counts().index[0]).reset_index()
    mode_features.columns = ['campaign_id', 'campaign_coupon_id', 'campaign_item_mode', 'campaign_brand_mode', 'campaign_category_mode']
    
    
    features = pd.merge(pd.merge(campaign, count_features, on = 'campaign_id', how = 'left'), mode_features, on = 'campaign_id', how = 'left')
    
    return features

In [17]:
campaign_features = campaign_feature(campaign, pd.merge(campaign_coupon, coupon, on = 'coupon_id', how = 'left'))

In [18]:
ccc_features, cx_features, cc_features  = txn_features(txn, campaign, coupon)

In [19]:
campaign_overlap_features = campaign_overlap(campaign[['campaign_id', 'campaign_type', 'start_date', 'end_date']])

In [20]:
customer_coupon_count_tr = train.groupby(['customer_id','campaign_id'])['coupon_id'].count().reset_index()
customer_coupon_count_tr.columns = ['customer_id','campaign_id', 'customer_campaign_coupon_cnt']
customer_coupon_count_ts = test.groupby(['customer_id','campaign_id'])['coupon_id'].count().reset_index()
customer_coupon_count_ts.columns = ['customer_id','campaign_id', 'customer_campaign_coupon_cnt']

In [21]:
tr = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(train, ccc_features.fillna(0), on = ['campaign_id','coupon_id','customer_id'], how = 'left'),
                  cx_features, on = ['campaign_id','customer_id'], how = 'left'), 
         cc_features, on = ['coupon_id','campaign_id'], how = 'left'),
                       campaign_features, on = ['campaign_id'], how = 'left'),
              campaign_overlap_features, on = ['campaign_id'], how = 'left'), 
                       customer, on = ['customer_id'], how = 'left'), customer_coupon_count_tr, on = ['customer_id','campaign_id'], how = 'left')

ts = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(test, ccc_features.fillna(0), on = ['campaign_id','coupon_id','customer_id'], how = 'left'),
                  cx_features, on = ['campaign_id','customer_id'], how = 'left'), 
         cc_features, on = ['coupon_id','campaign_id'], how = 'left'),
                       campaign_features, on = ['campaign_id'], how = 'left'),
              campaign_overlap_features, on = ['campaign_id'], how = 'left'), 
                       customer, on = ['customer_id'], how = 'left'), customer_coupon_count_ts, on = ['customer_id','campaign_id'], how = 'left')

In [22]:
def ratio(df):
    
    df['ratio_brand_cnt'] = df['ccc_Established_brand_cnt']/df['cx_Established_brand_cnt']
    df['ratio_Established_coupon_disc_per'] = df['ccc_Established_coupon_disc_per']/df['cx_Established_coupon_disc_per']
    df['ratio_Established_coupon_used'] = df['ccc_Established_coupon_used']/df['cx_Established_coupon_used']
    df['ratio_Established_gmv'] = df['ccc_Established_gmv']/df['cx_Established_gmv']
    df['ratio_Established_item_cnt'] = df['ccc_Established_item_cnt']/df['cx_Established_item_cnt']
    df['ratio_Established_order_cnt'] = df['ccc_Established_order_cnt']/df['cx_Established_order_cnt']
    df['ratio_Established_other_disc_per'] = df['ccc_Established_other_disc_per']/df['cx_Established_other_disc_per']
    df['ratio_Local_brand_cnt'] = df['ccc_Local_brand_cnt']/df['cx_Local_brand_cnt']
    df['ratio_Local_coupon_disc_per'] = df['ccc_Local_coupon_disc_per']/df['cx_Local_coupon_disc_per']
    df['ratio_Local_coupon_used'] = df['ccc_Local_coupon_used']/df['cx_Local_coupon_used']
    df['ratio_Local_gmv'] = df['ccc_Local_gmv']/df['cx_Local_gmv']
    df['ratio_Local_item_cnt'] = df['ccc_Local_item_cnt']/df['cx_Local_item_cnt']
    df['ratio_Local_order_cnt'] = df['ccc_Local_order_cnt']/df['cx_Local_order_cnt']
    df['ratio_Local_other_disc_per'] = df['ccc_Local_other_disc_per']/df['cx_Local_other_disc_per']
    df['ratio_total_qty'] = df['ccc_total_qty']/df['cx_total_qty']
    
    return df

In [23]:
tr = ratio(tr)
ts = ratio(ts)

In [24]:
drop_cols = ['start_date', 'end_date', 'id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status', 
             'first_txn_date', 'last_txn_date', 'campaign_item_mode', 'campaign_coupon_id', 'campaign_brand_mode']
cat_cols = ['campaign_type','campaign_start_month','campaign_brand_mode', 'campaign_category_mode', 
            'cc_frequent_cat_Estabilished', 'cc_frequent_cat_Local', 
            'age_range', 'marital_status', 'rented', 'family_size','no_of_children', 'income_bracket','coupon_id',
    'customer_id']
feature_cols = [
    'coupon_id',
    'customer_id',
'ccc_Established_brand_cnt',
'ccc_Established_coupon_affinity',
'ccc_Established_coupon_disc',
'ccc_Established_coupon_disc_per',
'ccc_Established_coupon_used',
'ccc_Established_gmv',
'ccc_Established_item_cnt',
'ccc_Established_item_cnt_per',
'ccc_Established_order_cnt',
'ccc_Established_other_disc',
'ccc_Established_other_disc_per',
'ccc_Local_brand_cnt',
'ccc_Local_coupon_affinity',
'ccc_Local_coupon_disc',
'ccc_Local_coupon_disc_per',
'ccc_Local_coupon_used',
'ccc_Local_gmv',
'ccc_Local_item_cnt',
'ccc_Local_order_cnt',
'ccc_Local_other_disc',
'ccc_Local_other_disc_per',
'ccc_qty_per_Bakery',
'ccc_qty_per_Dairy, Juices & Snacks',
'ccc_qty_per_Flowers & Plants',
'ccc_qty_per_Garden',
'ccc_qty_per_Grocery',
'ccc_qty_per_Meat',
'ccc_qty_per_Miscellaneous',
'ccc_qty_per_Natural Products',
'ccc_qty_per_Packaged Meat',
'ccc_qty_per_Pharmaceutical',
'ccc_qty_per_Prepared Food',
'ccc_qty_per_Restauarant',
'ccc_qty_per_Salads',
'ccc_qty_per_Seafood',
'ccc_qty_per_Skin & Hair Care',
'ccc_qty_per_Travel',
'ccc_qty_per_Vegetables (cut)',
'ccc_total_qty',
'cx_Established_item_cnt',
'cx_Local_item_cnt',
'cx_Established_item_cnt_per',
'cx_weekday_qty_cnt',
'cx_weekend_qty_cnt',
'cx_weekend_affinity',
'cx_Established_brand_cnt',
'cx_Local_brand_cnt',
'cx_Established_gmv',
'cx_Local_gmv',
'cx_Established_gmv_per',
'cx_Established_other_disc',
'cx_Local_other_disc',
'cx_Established_coupon_disc',
'cx_Local_coupon_disc',
'cx_total_qty',
'cx_qty_per_Alcohol',
'cx_qty_per_Bakery',
'cx_qty_per_Dairy, Juices & Snacks',
'cx_qty_per_Flowers & Plants',
'cx_qty_per_Fuel',
'cx_qty_per_Garden',
'cx_qty_per_Grocery',
'cx_qty_per_Meat',
'cx_qty_per_Miscellaneous',
'cx_qty_per_Natural Products',
'cx_qty_per_Packaged Meat',
'cx_qty_per_Pharmaceutical',
'cx_qty_per_Prepared Food',
'cx_qty_per_Restauarant',
'cx_qty_per_Salads',
'cx_qty_per_Seafood',
'cx_qty_per_Skin & Hair Care',
'cx_qty_per_Travel',
'cx_qty_per_Vegetables (cut)',
'cx_Established_coupon_used',
'cx_Local_coupon_used',
'cx_Established_order_cnt',
'cx_Local_order_cnt',
'cx_Established_coupon_affinity',
'cx_Local_coupon_affinity',
'cx_Established_other_disc_per',
'cx_Established_coupon_disc_per',
'cx_Local_other_disc_per',
'cx_Local_coupon_disc_per',
'Vintage',
'days_since_last_txn',
'cc_item_count_Estabilished',
'cc_item_count_Local',
'cc_brand_count_Estabilished',
'cc_brand_count_Local',
'cc_frequent_cat_Estabilished',
'cc_frequent_cat_Local',
'cc_avg_asp_Estabilished',
'cc_avg_asp_Local',
'cc_coupon_discount_Estabilished',
'cc_coupon_discount_Local',
'cc_customers_Estabilished',
'cc_customers_Local',
'cc_coupon_used_Established',
'cc_coupon_used_Local',
'campaign_type',
'campaign_days',
'campaign_weekends',
'campaign_start_month',
'campaign_unique_coupon_cnt',
'campaign_unique_item_cnt',
'campaign_unique_brand_cnt',
'campaign_unique_category_cnt',
'campaign_coupon_id',
'campaign_item_mode',
'campaign_brand_mode',
'campaign_category_mode',
'x_overlap_count',
'y_overlap_count',
'x_max_overlap_days',
'y_max_overlap_days',
'age_range',
'marital_status',
'rented',
'family_size',
'no_of_children',
'income_bracket',
'ratio_brand_cnt',
'ratio_Established_coupon_disc_per',
'ratio_Established_coupon_used',
'ratio_Established_gmv',
'ratio_Established_item_cnt',
'ratio_Established_order_cnt',
'ratio_Established_other_disc_per',
'ratio_Local_brand_cnt',
'ratio_Local_coupon_disc_per',
'ratio_Local_coupon_used',
'ratio_Local_gmv',
'ratio_Local_item_cnt',
'ratio_Local_order_cnt',
'ratio_Local_other_disc_per',
'ratio_total_qty']
drop_features = ['income_bracket',
                'family_size',
                 'rented',
                 'cc_avg_asp_Estabilished',
                 'ccc_Established_gmv',
                 'cx_qty_per_Prepared Food',
                 'cx_Established_brand_cnt',
                 'campaign_unique_brand_cnt',
                 'cx_Established_coupon_disc',
                 'coupon_id','customer_id',
                 #'campaign_start_month',

                 'x_max_overlap_days'
                 #'ccc_qty_per_Bakery'
                 #'ratio_total_qty',
                 #'ratio_Established_coupon_used'
                 #'ratio_Established_order_cnt'#,
                 #'cx_Established_gmv_per',
                 #'campaign_unique_category_cnt'
                 #'cc_avg_asp_Local'
                 #'ccc_Established_item_cnt'
                 #'ratio_Local_other_disc_per'
                ]

In [25]:
for col in cat_cols:
    tr[col] = tr[col].astype('category')
    ts[col] = ts[col].astype('category')

In [26]:
tr2 = tr

In [27]:
dev, oot = train_test_split(tr2, stratify=tr2.campaign_id, test_size=0.20, random_state=444)
dev, val = train_test_split(dev, stratify=dev.campaign_id, test_size=0.20, random_state=1)

In [28]:
params = {"objective": "binary",
            "max_bin": 50,
            "metric": "auc",
            "boosting": "gbdt",
            "num_leaves": 7,
            "max_depth": -1,
            "min_child_weight": 60,
            "learning_rate": 0.01,
            "bagging_fraction": 0.5,
            "feature_fraction": 0.4,
            "bagging_frequency": 5,
            "bagging_seed": 101,
            "random_seed": 101}
#.drop(columns = drop_features)
lgdev = lgb.Dataset(dev[feature_cols].drop(columns = drop_features), label=dev.redemption_status)
lgval = lgb.Dataset(val[feature_cols].drop(columns = drop_features), label=val.redemption_status)
lgoot = lgb.Dataset(oot[feature_cols].drop(columns = drop_features), label=oot.redemption_status)
    
evals_result = {}
model = lgb.train(params, lgdev, 20000, valid_sets=[lgval], early_stopping_rounds=50, verbose_eval=None, evals_result=evals_result)
    
pred_dev_y = model.predict(dev[feature_cols].drop(columns = drop_features), num_iteration=model.best_iteration)
pred_val_y = model.predict(val[feature_cols].drop(columns = drop_features), num_iteration=model.best_iteration)
pred_oot_y = model.predict(oot[feature_cols].drop(columns = drop_features), num_iteration=model.best_iteration)
pred_ts = model.predict(ts[feature_cols].drop(columns = drop_features), num_iteration=model.best_iteration)

In [29]:
print(roc_auc_score(dev.redemption_status, pred_dev_y))
print(roc_auc_score(val.redemption_status, pred_val_y))
print(roc_auc_score(oot.redemption_status, pred_oot_y))

0.963182424304
0.932139756195
0.930019897016


In [ ]:
submission = pd.DataFrame({"id":ts.id, 'redemption_status':pred_ts})
submission.to_csv("ultimate.csv", index = False)

In [ ]:
result = pd.DataFrame()
#.drop(columns = drop_features).columns
for c in tr[feature_cols].drop(columns = drop_features).columns:
    #c = ['unique_coupon_brand']
    lgdev = lgb.Dataset(dev[feature_cols].drop(columns = drop_features).drop(columns = c), label=dev.redemption_status)
    lgval = lgb.Dataset(val[feature_cols].drop(columns = drop_features).drop(columns = c), label=val.redemption_status)

    evals_result = {}
    model = lgb.train(params, lgdev, 20000, valid_sets=[lgval], early_stopping_rounds=50, verbose_eval=None, evals_result=evals_result)

    pred_dev_y = model.predict(dev[feature_cols].drop(columns = drop_features).drop(columns = c), num_iteration=model.best_iteration)
    pred_val_y = model.predict(val[feature_cols].drop(columns = drop_features).drop(columns = c), num_iteration=model.best_iteration)
    pred_oot_y = model.predict(oot[feature_cols].drop(columns = drop_features).drop(columns = c), num_iteration=model.best_iteration)
    
    result.loc[c,'dev_auc'] = roc_auc_score(dev.redemption_status, pred_dev_y)
    result.loc[c,'val_auc'] = roc_auc_score(val.redemption_status, pred_val_y)
    result.loc[c,'oot_auc'] = roc_auc_score(oot.redemption_status, pred_oot_y)
    
    print("Columns", c)
    print("dev auc :",roc_auc_score(dev.redemption_status, pred_dev_y))
    print("val auc :",roc_auc_score(val.redemption_status, pred_val_y))
    print("oot auc :",roc_auc_score(oot.redemption_status, pred_oot_y))
    print("")